# Using Python, requests and Pandas

[Python](https://www.python.org) is a popular programming language which is heavily used in the data science domains.  Python provides high level functionality supporting rapid application development with a large ecosystem of packages to work with weather/climate/water data.

Let's use the [Python requests](https://docs.python-requests.org) package to further interact with the wis2box API, and [Pandas](https://pandas.pydata.org) to run some simple summary statistics.

In [1]:
import json

import requests

def pretty_print(input):
    print(json.dumps(input, indent=2))


# define the endpoint of the OGC API
api = 'http://localhost:8999/oapi'

## Stations

Let's find all the stations in our wis2box:

In [2]:
url = f'{api}/collections/stations/items?limit=50'

response = requests.get(url).json()

print(f"Number of stations: {response['numberMatched']}")

print('Stations:\n')
for station in response['features']:
    print(station['properties']['name'])

Number of stations: 26
Stations:

NAMBUMA
BALAKA
BILIRA
CHIDOOLE
CHIKANGAWA
CHIKWEO
CHINGALE
KALAMBO
KASIYA AWS
KASUNGU NATIONAL PARK AWS
KAWALAZI
KAYEREKERA
LENGWE NATIONAL PARK
LOBI AWS
MAKANJIRA
MALOMO
MISUKU
MLARE
MLOMBA
MTOSA BENGA
NAMITAMBO
NANKUMBA
NKHOMA UNIVERSITY
NKHULAMBE
NYACHILENDA
TOLEZA


## Discovery Metadata

Now, let's find all the dataset that are provided by the above stations.  Each dataset is identified by a WIS 2.0 discovery metadata record.

In [3]:
url = f'{api}/collections/discovery-metadata/items'

response = requests.get(url).json()

print('Datasets:\n')
for dataset in response['features']:
    print(f"id: {dataset['properties']['id']}, title: {dataset['properties']['title']}")

Datasets:

id: data.core.test-passthrough, title: Surface weather observations (passthrough)
id: data.core.observations-surface-land.mw.FWCL.landFixed, title: Surface weather observations (hourly)


Let's find all the data access links associated with the Surface weather observations (hourly) dataset:

In [4]:
dataset_id = 'data.core.observations-surface-land.mw.FWCL.landFixed'

url = f"{api}/collections/discovery-metadata/items/{dataset_id}"

response = requests.get(url).json()

print('Data access links:\n')
for link in response['links']:
    print(f"{link} {link['href']} ({link['type']}) {link['rel']}")
    link['rel']

[link['href'] for link in response['links']]

Data access links:

{'rel': 'self', 'type': 'application/geo+json', 'title': 'This document as GeoJSON', 'href': 'http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=json'} http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=json (application/geo+json) self
{'rel': 'alternate', 'type': 'application/ld+json', 'title': 'This document as RDF (JSON-LD)', 'href': 'http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=jsonld'} http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=jsonld (application/ld+json) alternate
{'rel': 'alternate', 'type': 'text/html', 'title': 'This document as HTML', 'href': 'http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=html'} ht

['http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=json',
 'http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=jsonld',
 'http://localhost:8999/oapi/collections/discovery-metadata/items/data.core.observations-surface-land.mw.FWCL.landFixed?f=html',
 'http://localhost:8999/oapi/collections/discovery-metadata']

Let's use the OGC API - Features (OAFeat) link to drill into the observations for Chidoole station

In [5]:
dataset_api_link = 'http://localhost:8999/oapi/collections/data.core.observations-surface-land.mw.FWCL.landFixed'

dataset_api_link

'http://localhost:8999/oapi/collections/data.core.observations-surface-land.mw.FWCL.landFixed'

## Observations

Let's inspect some of the data in the API's raw GeoJSON format:

In [6]:
url = f'{dataset_api_link}/items'

query_parameters = {
    'wigos_station_identifier': '0-454-2-AWSCHIDOOLE',
    'limit': 10000,
    'name': 'air_temperature'
}

response = requests.get(url, params=query_parameters).json()

pretty_print(response['features'][0])

{
  "id": "WIGOS_0-454-2-AWSCHINGALE_20220112T135500-25",
  "reportId": "WIGOS_0-454-2-AWSCHINGALE_20220112T135500",
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      35.11,
      -15.24,
      623.0
    ]
  },
  "properties": {
    "wigos_station_identifier": "0-454-2-AWSCHINGALE",
    "phenomenonTime": "2022-01-12T13:55:00Z",
    "resultTime": "2022-01-12T13:55:00Z",
    "name": "air_temperature",
    "value": 24.85,
    "units": "Celsius",
    "description": null,
    "metadata": [
      {
        "name": "station_or_site_name",
        "value": null,
        "units": "CCITT IA5",
        "description": "Chingale"
      },
      {
        "name": "station_type",
        "value": 0,
        "units": "CODE TABLE",
        "description": "Automatic"
      },
      {
        "name": "height_of_barometer_above_mean_sea_level",
        "value": 624.0,
        "units": "m",
        "description": null
      },
      {
        "name": "height_of_sensor_abo

Let's inspect what's measured at Chidoole:

In [7]:
print('Observed property:\n')
feature = response['features'][9]
print(f"{feature['properties']['name']} ({feature['properties']['units']})")

Observed property:

air_temperature (Celsius)


## Pandas
Let's use the GeoJSON to build a more user-friendly table

In [8]:
import pandas as pd

datestamp = [obs['properties']['resultTime'] for obs in response['features']]
air_temperature = [obs['properties']['value'] for obs in response['features']]

d = {
    'Date/Time': datestamp,
    'Air temperature (°C)': air_temperature
}

df = pd.DataFrame(data=d)

In [9]:
df

,Date/Time,Air temperature (°C)
0,2022-01-12T13:55:00Z,24.85
1,2022-01-12T14:55:00Z,27.25
2,2022-01-12T15:55:00Z,26.65
3,2022-01-12T16:55:00Z,25.95
4,2022-01-12T17:55:00Z,25.45
...,...,...
5101,2022-06-09T12:55:00Z,21.35
5102,2022-06-09T13:55:00Z,22.25
5103,2022-06-09T14:55:00Z,20.25
5104,2022-06-10T12:55:00Z,23.75


In [10]:
print("Time extent\n")
print(f'Begin: {df["Date/Time"].min()}')
print(f'End: {df["Date/Time"].max()}')

print("Summary statistics:\n")
df[['Air temperature (°C)']].describe()

Time extent

Begin: 2022-01-12T13:55:00Z
End: 2022-06-10T14:55:00Z
Summary statistics:



,Air temperature (°C)
count,5106.000000
mean,23.541559
std,4.053172
min,13.550000
25%,20.950000
50%,23.350000
75%,26.350000
max,37.850000
